In [1]:
import base64
import datetime
from urllib.parse import urlencode

import requests

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):

        #Returns a base64 encoded string

        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }

    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client")
            #return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token

    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers

    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query,"type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [2]:
client_id = 'f7eedef5bb4b4fe4ad5f7b276f8db10c'
client_secret = '14505366a61b431994f7afe58ecdc550'
user_id = "1113039340"

spotify = SpotifyAPI(client_id, client_secret)

In [5]:
import  pandas as pd

Data = spotify.search({"artist": "Drake"}, search_type="track")

need = []
for i, item in enumerate(Data['tracks']['items']):
    track = item['album']
    track_id = item['id']
    song_name = item['name']
    popularity = item['popularity']
    need.append((i, track['artists'][0]['name'], track['name'], track_id, song_name, track['release_date'], popularity))

Track_df = pd.DataFrame(need, index=None, columns=('Item', 'Artist', 'Album Name', 'Id', 'Song Name', 'Release Date', 'Popularity'))

q=artist%3ADrake&type=track
https://api.spotify.com/v1/search?q=artist%3ADrake&type=track


In [39]:
a = spotify.get_access_token()

In [40]:
import spotipy

In [41]:
a

'BQD_Y3wmuckTARymIKz9nEDdF3-PcA1AOULORLzp2tWN-CuoxUpToeM2A2w-mSpuRZ6sNIFYV79rRgnMmJU'

In [31]:
def get_playist_id_from_link(playlist_link):
    """
    Function returns the Spotify playlist id from the Spotify generated link
    :param playlist_link:
    :return:
    """
    id = playlist_link.split("/")[-1].split("?")[0]
    return id

In [32]:
id = get_playist_id_from_link("https://open.spotify.com/playlist/37i9dQZF1EJDvX9MsK3miT?si=58158136514a439a")

In [33]:
id

'37i9dQZF1EJDvX9MsK3miT'

In [28]:
sp = spotipy.Spotify(auth=spotify.access_token)

In [29]:
sp

In [48]:
playlist = sp.playlist(user_id, "37i9dQZF1EJDvX9MsK3miT")
name = playlist["name"]

2022-02-06 19:12:48.169 ERROR   spotipy.client: HTTP Error for GET to https://api.spotify.com/v1/playlists/1113039340 with Params: {'fields': '37i9dQZF1EJDvX9MsK3miT', 'market': None, 'additional_types': 'track'} returned 404 due to Invalid playlist Id


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/playlists/1113039340?fields=37i9dQZF1EJDvX9MsK3miT&additional_types=track:
 Invalid playlist Id, reason: None

In [49]:
token = util.prompt_for_user_token(
user_id, SPOTIPY_SCOPE, client_id = client_id, client_secret = client_secret, redirect_uri='http://localhost/')
sp = spotipy.Spotify(auth = token)

In [44]:
import streamlit as st
import pandas as pd
import numpy as np
import spotipy
import spotipy.util as util
from dateutil.parser import parse as parse_date
import plotly.graph_objects as go
from pylab import *
import calplot

################################################## VARIABLES ###########################################################

FILE_PATH_API_CREDENTIAL = r"C:\Users\Jaume\Documents\Python Projects\spotify_compare\data\external\api_keys.csv"
SPOTIPY_SCOPE = "ugc-image-upload user-read-playback-state streaming user-read-email playlist-read-collaborative \
    user-modify-playback-state user-read-private playlist-modify-public user-library-modify user-top-read \
    user-read-playback-position user-read-currently-playing playlist-read-private user-follow-read \
    app-remote-control user-read-recently-played playlist-modify-private user-follow-modify user-library-read"
ARTIST = "artist"